# Creación de un agente que hable como si fuera yo, con datos de linkedin

No usa RAG, es una práctica de SYSTEM_PROMPT

## Importaciones

In [1]:
!pip install openai
!pip install gradio
!pip install PyPDF2

In [2]:
from openai import OpenAI
from typing import List, Dict, Optional
from IPython.display import display, Markdown
import gradio as gr
from PyPDF2 import PdfReader

c:\Users\Jorge\Desktop\programacion\CURSO\agents_frogames_my_examples\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Funciones

In [3]:
class Llm:
    def __init__(self, model: str, url: str, api_key: str, system_prompt: str = ""):
        self._model = model
        self._url = url
        self._api_key = api_key
        self._system_prompt = system_prompt
        self._client = self._load_openai()

    @property
    def model(self) -> str:
        return self._model

    @model.setter
    def model(self, value: str):
        self._model = value

    @property
    def url(self) -> str:
        return self._url

    @url.setter
    def url(self, value: str):
        self._url = value
        self._client = self._load_openai()  
    
    @property
    def system_prompt(self) -> str:
        return self._system_prompt
    
    @system_prompt.setter
    def system_prompt(self, new_sys_prompt: str):
        self._system_prompt = new_sys_prompt

    def _load_openai(self) -> OpenAI:
        return OpenAI(base_url=self._url, api_key=self._api_key)

    def _format_message(self, user_message: str) -> List[Dict[str, str]]:
        return [{"role": "user", "content": user_message}]
    
    def _format_sys_prompt(self) -> List[Dict[str, str]]:
        return [{"role": "system", "content": self._system_prompt}]

    def generate_response(self, user_message: str) -> str:
        if not self._system_prompt:
            messages = self._format_message(user_message)
        else:
            messages = self._format_sys_prompt() + self._format_message(user_message)
        
        response = self._client.chat.completions.create(
            model=self._model,
            messages=messages
        )
        return response.choices[0].message.content

    def chat(self, message, history):
        messages = self._format_sys_prompt() + history + self._format_message(message)
        response = self._client.chat.completions.create(
            model=self._model,
            messages=messages
        )
        return response.choices[0].message.content

    def visualize_response(self, response: str) -> None:
        display(Markdown(response))
    
    @staticmethod
    def launch_chat(chat):
        gr.ChatInterface(fn=chat, type="messages").launch(share=True)


class PdfParser:
    def __init__(self, route: str):
        self._route = route
        self._reader = self._load_reader()

    def _load_reader(self) -> Optional[PdfReader]:
        try:
            return PdfReader(self._route)
        except Exception as e:
            print(f"Problems loading pdf: {e}")
            return None

    def read_document(self) -> str:
        if self._reader is None:
            return ""
        text_return = ""
        for page in self._reader.pages:
            text = page.extract_text()
            if text:
                text_return += text
        return text_return

## Código principal

In [4]:
#Primero usa ollama serve en CMD, si no está descargado ollama, descomentar
#!ollama run llama3.2

Cargamos el pdf

In [5]:
pdfParser = PdfParser("about_me/linkedin.pdf")
linkedin_doc = pdfParser.read_document()

Creamos el System_prompt

In [ ]:
name = "Jorge Sequeiros"

system_prompt = f"Estás actuando como {name}. Estás respondiendo preguntas en el sitio web de {name}, en particular preguntas relacionadas con la carrera, la trayectoria, las habilidades y la experiencia de {name}. \
Tu responsabilidad es representar a {name} en las interacciones en el sitio web con la mayor fidelidad posible. \
Se te proporciona el perfil de LinkedIn de {name} que puedes usar para responder preguntas. \
Sé profesional y atractivo, como si hablaras con un cliente potencial o un futuro empleador que haya visitado el sitio web. \
Si no sabes la respuesta, dilo."

system_prompt += f"## Perfil de LinkedIn:\n{linkedin_doc}\n\n"
system_prompt += f"En este contexto, charla con el usuario, utilizando siempre el personaje de {name}."

Cargamos el modelo y lo utilizamos

In [7]:
model_name = "llama3.2"
llm = Llm(model=model_name, url='http://localhost:11434/v1', api_key="ollama", system_prompt=system_prompt)

Lanzamos interfaz

In [8]:
llm.launch_chat(llm.chat)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://7ee6b1ccf258b2b13e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
